In [342]:
# Use the most gernal way to extract data from XML with different liquidity horizon
def FRTB_read_xml(file):
    import numpy as np
    import pandas as pd
    content = open(file).read()
    ind_riskclass_s = content.index('<pvRiskClassList>')
    ind_riskclass_e = content.index('</pvRiskClassList>')
    
    ind_currentdays = content.index('<date>')
    ind_currentdaye = content.index('</date>')
    currentday = content[ind_currentdays+len('<date>'):ind_currentdaye]
    riskclass = content[ind_riskclass_s+len('<pvRiskClassList>'): ind_riskclass_e].split(' ')
    risktypes_num = len(riskclass)
    ind_ds= content.index('<scenarioDates>')
    ind_de = content.index('</scenarioDates>')
    Dates = content[ind_ds+len('<scenarioDates>'):ind_de].split(' ')
    def find_substring(substring, string):
        indices = []
        index = -1  # Begin at -1 so index + 1 is 0
        while True:
            # Find next index of substring, by starting search from index + 1
            index = string.find(substring, index + 1)
            if index == -1:  
                break  # All occurrences have been found
            indices.append(index)
        return indices
    def read_data(demo,num_risks):
        ind_pvs = find_substring('<pvList>',demo)
        ind_pve = find_substring('</pvList>',demo)
        ind_ns = find_substring('<name>',demo)
        ind_ne = find_substring('</name>',demo)
        ind_vs = find_substring('<value>',demo)
        ind_ve = find_substring('</value>',demo)
        ind_currentpvs = demo.index('<currentPv>')
        ind_currentpve = demo.index('</currentPv>')
 
        currentpv = demo[ind_currentpvs+len('<currentPv>'):ind_currentpve]
        names = []
        for i in range(len(ind_ns)):
            names.append(demo[ind_ns[i]+len('<name>'):ind_ne[i]])
        names.append('current_value')

        values = []
        for i in range(len(ind_vs)):
            values.append(demo[ind_vs[i]+len('<value>'):ind_ve[i]])
        values.append(currentpv)

        pv = []
        for i in range(num_risks):
            pv.append(demo[(ind_pvs[i]+len('<pvList>')):ind_pve[i]].split(' '))
        
        return pv,values
    
    
    trades = content.split('<externalId>')[1:]


    trades_info = []
    trades_pv = []

    for i in range(len(trades)):
        trades_pv.append(read_data(trades[i], risktypes_num)[0])
        trades_info.append(read_data(trades[i], risktypes_num)[1])
        
        
    PV = np.array(trades_pv).astype('f')
    pvlist = PV[0].reshape((len(Dates),len(riskclass)))
    for i in range(1,len(PV)):
        pvlist = np.hstack([pvlist,PV[i].reshape((len(Dates),len(riskclass)))])
    tradeid = [ i[-3] for i in trades_info]

    columns = pd.MultiIndex.from_product([tradeid,riskclass],names = ['tradeid','riskclass'])
    PVLIST = pd.DataFrame(pvlist,index=Dates,columns=columns)
    PVLIST.index = pd.to_datetime(PVLIST.index)
    PL= PVLIST.diff()
    PL= PL.drop(PL.index[0])
    
    
    
    
    
    
    
    return PL, trades_info 

    
# The output is P&L dataframe showed below and each trade information    

In [343]:
PL_10D_pd = FRTB_read_xml("hvar_RGRP_Sample_10D.xml")[0]
PL_20D_pd = FRTB_read_xml("hvar_RGRP_Sample_20D.xml")[0]
PL_40D_pd = FRTB_read_xml("hvar_RGRP_Sample_40D.xml")[0]

In [344]:
PL_10D_pd.head()

tradeid        19858S                                      19859S             \
riskclass          IR         FX         EQ      TOTAL         IR         FX   
2018-07-20 -1947528.0 -1109552.0 -1022424.0  -535992.0 -1947568.0 -1109576.0   
2018-07-23   101968.0   -53344.0   337552.0   474072.0   101968.0   -53352.0   
2018-07-24  1520504.0  2513428.0  4030568.0  3489292.0  1520528.0  2513488.0   
2018-07-25  1266860.0  1501116.0  -231188.0  1275680.0  1266896.0  1501148.0   
2018-07-26  1298364.0   647640.0   443636.0 -2278812.0  1298392.0   647660.0   

tradeid                              19861S             ...  0000035642  \
riskclass          EQ      TOTAL         IR         FX  ...          EQ   
2018-07-20 -1022440.0  -536008.0 -3176176.0 -1703576.0  ...   582.37500   
2018-07-23   337552.0   474072.0    66168.0  -181584.0  ...  -808.15625   
2018-07-24  4030660.0  3489388.0  2614864.0  3996304.0  ...  1178.06250   
2018-07-25  -231192.0  1275704.0  1750904.0  2356608.0  ... -2118.71875   
2018-07-26   443644.0 -2278876.0  2183320.0  1001216.0  ...   848.93750   

tradeid                  19856S                           \
riskclass        TOTAL       IR      FX      EQ    TOTAL   
2018-07-20   630.09375  -1312.0  1560.0  5688.0  10232.0   
2018-07-23 -2253.53125  14264.0  2312.0  7472.0   8976.0   
2018-07-24  2284.06250   3552.0  5600.0 -4104.0  -9136.0   
2018-07-25 -1512.93750  -2080.0 -3560.0 -4288.0  -2352.0   
2018-07-26 -2661.93750  -5560.0 -2968.0  6248.0  17032.0   

tradeid    000000FQ|27/12/18|SUMMIT|EQUITY|VARSTT|                  
riskclass                                       IR   FX   EQ TOTAL  
2018-07-20                                     0.0  0.0  0.0   0.0  
2018-07-23                                     0.0  0.0  0.0   0.0  
2018-07-24                                     0.0  0.0  0.0   0.0  
2018-07-25                                     0.0  0.0  0.0   0.0  
2018-07-26                                     0.0  0.0  0.0   0.0  

[5 rows x 80 columns]

In [422]:
FRTB_read_xml("hvar_RGRP_Sample_40D.xml")[1]

[['VARSTT', 'SUMMIT', 'JPM', 'RATES', 'USD', '19858S', 'SWAP', '0.0'],
 ['VARSTT', 'SUMMIT', 'JPM', 'RATES', 'USD', '19859S', 'SWAP', '0.0'],
 ['VARSTT', 'SUMMIT', 'MS', 'RATES', 'USD', '19861S', 'SWAP', '0.0'],
 ['VARSTT', 'SUMMIT', 'MS', 'RATES', 'USD', '19862S', 'SWAP', '0.0'],
 ['VARSTT', 'SUMMIT', 'JPM', 'RATES', 'USD', '19867S', 'SWAP', '0.0'],
 ['VARSTT', 'SUMMIT', 'MS', 'RATES', 'USD', '19868S', 'SWAP', '0.0'],
 ['VARSTT',
  'SUMMIT',
  ' ',
  'IRFUT',
  'USD',
  'OE_3MUSD|SEP20   |SUMMIT|IRFUT|VARSTT|',
  'FUT',
  '0.0'],
 ['VARSTT', 'SUMMIT', '_SSBT_LON', 'FX', 'USD', '0000035632', 'FXFWD', '0.0'],
 ['VARSTT', 'SUMMIT', '_GS_LON', 'FX', 'USD', '0000035633', 'FXFWD', '0.0'],
 ['VARSTT', 'SUMMIT', '_SST_FUND1', 'FX', 'USD', '0000035634', 'FXFWD', '0.0'],
 ['VARSTT', 'SUMMIT', '_SST_FUND1', 'FX', 'USD', '0000035635', 'FXFWD', '0.0'],
 ['VARSTT', 'SUMMIT', '_SST_FUND1', 'FX', 'USD', '0000035636', 'FXFWD', '0.0'],
 ['VARSTT', 'SUMMIT', '_SSBT_LON', 'FX', 'USD', '0000035637', 'FXFW

In [365]:
#get ES from X-days liquidity horizon P&L dataframe
def get_ES(pl):
    
    pl_pd = pl.xs('TOTAL',level='riskclass',axis=1)
    pl = np.sum(pl_pd,axis=1).values
    num = np.floor(len(pl_portfolio)*0.025)
    ES = np.mean(sorted(pl)[:int(num)])
    return ES
        
   

In [366]:
get_ES(PL_10D_pd), get_ES(PL_20D_pd), get_ES(PL_40D_pd)

(-38166880.0, -76926.0, -23372.0)

In [370]:
#ES_cal function is to get the ajusted ES, added with liquidity horizon
#Input is general P&L, P&L with differnet LH, the arguments is felixible
def ES_cal( pl_total,pl_10,*pl_others):
    ES = get_ES(pl_total)
    
    if pl_others is not None:
        T = len(pl_others[0])/250
        ES_days = [0,0,0,0]
        for i in range(len(pl_others)):
            ES_days[i] = get_ES(pl_others[i])
            
        LH = [np.sqrt(10/T), np.sqrt(20/T), np.sqrt(20/T), np.sqrt(60/T)]
            
        ES1 = np.sqrt(ES**2 + (ES_days[0]*LH[0])**2+(ES_days[1]*LH[1])**2+(ES_days[2]*LH[2])**2+(ES_days[3]*LH[3])**2)
        return ES1
    else:
        return ES
        
    
        

In [371]:

#Here I tried the 3 files you give us
#Take P&L with 10 LH as general one (or total P&L), we check the answer is quite nice

ES_cal(PL_10D_pd,PL_10D_pd, PL_20D_pd,PL_40D_pd)

38169749.735615574

IMCC = 0.5*IMCC(C) + 0.5*SUM(IMCC(Ci))

IMCC(C) = ES_RedS*(ES_fullc/ES_redc)
IMCC(Ci) = ES_RedSi*(ES_fullc_i/ES_redc_i)

In [420]:

#this function get the partial P&L accroding 5 different risk classes
#input one portfolio P&L dataframe, output is partial P&L according to 5 classes of risk
def get_ES_partial(pl):
    
    pl_pd = [0,0,0,0,0]
    ES_i = [0,0,0,0,0]
    riskfac = ['IR','FX','EQ','CO','CS']
    #there are five classes of risks, the last two are commodity risk and credit spread risk
    for i in range(len(riskfac)) :
        if ( riskfac[i] in pl.columns.get_level_values(1)):
            print( riskfac[i])
            pl_pd[i] =  pl.xs(riskfac[i],level='riskclass',axis=1)
            pl_partial = np.sum(pl_pd[i], axis=1).values
            num = np.floor(len(pl_partial)*0.025)
            ES_i[i] = np.mean(sorted(pl_partial)[:int(num)])
            print(ES_i[i])
            
    return ES_i
        
                

In [421]:
get_ES_partial(PL)
#these are P&L according to 5 classes of risk

IR
-33257124.0
FX
-41201070.0
EQ
-201725710.0


[-33257124.0, -41201070.0, -201725710.0, 0, 0]

In [ ]:
#this function calculate IMCC, the input is P&L dataframe with reduced risk factors
# in stress time/ reduced risk factor in current time / full risk factor in current time
# Output is IMCC value
def Cal_IMCC(pl_rs,pl_rc,pl_fc):
    ES_rs = get_ES(pl_rs)
    ES_rc = get_ES(pl_rc)
    ES_fc = get_ES(pl_fc)
    ES_rs_partial = get_ES_partial(pl_rs)
    ES_rc_partial = get_ES_partial(pl_rc)
    ES_fc_partial = get_ES_partial(pl_fc)
    
    IMCC_c = ES_rs*(ES_fc/ES_rc)
    IMCC_ci = [0,0,0,0,0]
    for i in range(5):
        if (ES_rs_partial!=0 and ES_rc_partial!=0 and ES_fc_partial!=0):
            IMCC_ci[i] = ES_rs_partial*(ES_fc_partial/ES_rc_partial)
            
    IMCC = 0.5*IMCC_c + 0.5*sum(IMCC_ci)
    
    return IMCC
            
        
   